# Python 網頁爬蟲與資料視覺化應用 Final Project
## 成大通識與各系必修撞課分析
作者：物理系 施宇庭 C24066096

1. [問題定義](#問題定義)
2. [資料收集](#資料收集)
3. [資料前處理](#資料前處理)
4. [視覺化呈現](#視覺化呈現)
5. [結果分析與討論](#結果分析與討論)

## 問題定義
[回頂部](#成大通識與各系必修撞課分析)

選課季節又到了，，但因為和系上必修撞課，很難選到又甜又涼的通識，，，因此，這次期末專題打算來做各系必修與通識的撞課情況分析

統計各個科系必修的上課時間中有多少通識撞課，

備註：研究所不需要修通識課，因此這份報告將專注在大學部的部分

希望藉由這份分析找出，提供各系及通識中心排課參考，以平衡各系學生選擇通識課程的權益

## 資料收集
資料來源：[NCKU Open Data Platform](http://data.ncku.edu.tw/dataset/8d6462ee513f1c1a601506265cc9f674)

可從上方連結下載資料，或是直接執行專案目錄下的 `get_data.sh` 來下載資料，預設儲存路徑為 `data/`
```bash
source get_data.sh [儲存路徑]
```

**注意：若自行給定下載路徑，則下方程式碼的路徑也要記得修改**

[回頂部](#成大通識與各系必修撞課分析)

匯入所需套件

In [2]:
import pandas as pd

載入資料並觀察，由於資料的完整性，我們先拿106學年度第2學期的資料進行分析

In [107]:
path = './data/1062.csv'
df = pd.read_csv(path, encoding='cp950')
df.head(3)

,學年度,設立別,學校類別,學校代碼,學校名稱,學期別,開課單位,學制,課程代碼,課程名稱,...,結束時間,重複間隔,週次,課程類別,授課教師,學程名稱及類別,授課語言,上課地點,課程期程別,備註
0,106,公立,一般大學,5,國立成功大學,2,外國語言,大學部,A1107001,跨文化溝通,...,11:00,每週,NaN,通識(外國語言),鍾淑玫,NaN,中文,修齊大樓 26304,一般,模組一
1,106,公立,一般大學,5,國立成功大學,2,外國語言,大學部,A1106002,基礎學術溝通,...,11:00,每週,NaN,通識(外國語言),賴諭萱,NaN,中文,修齊大樓 26104,一般,模組一
2,106,公立,一般大學,5,國立成功大學,2,外國語言,大學部,A1105003,基礎學術英文,...,11:00,每週,NaN,通識(外國語言),辛佳玲,NaN,中文,修齊大樓 26105,一般,模組一


## 資料前處理
1. 查看資料中是否包含所有我們需要的欄位
2. 處理缺失值(missing value)
3. 處理重複值(duplicate value)
3. 最後刪除掉不必要的欄列資料

[回頂部](#成大通識與各系必修撞課分析)

先針對資料做更仔細的觀察，以下是重點關注的欄位

主要分析的欄位有5個：
- **開課單位**：判斷是否屬於必修或通識
- **星期**、**開始時間**、**結束時間**：判斷有無撞課
- **課程類別**：依照不同領域的通識做更細部的分析

幫助我們篩選資料的欄位有2個：
- **學制**：篩選出大學部的課程
- **必選修**：只保留必修部分(各系必修和通識都算必修)

其餘欄位都不重要，可以捨棄

> 備註：有些研究所會開課給大學部學生，例如大碩合開，因此以學制篩選資料，會看到開課單位還是有研究所

In [108]:
df.columns

Index(['學年度', '設立別', '學校類別', '學校代碼', '學校名稱', '學期別', '開課單位', '學制', '課程代碼',
       '課程名稱', '必選修', '學分數', '起始日期', '結束日期', '星期', '開始時間', '結束時間', '重複間隔',
       '週次', '課程類別', '授課教師', '學程名稱及類別', '授課語言', '上課地點', '課程期程別', '備註'],
      dtype='object')

檢查缺失值，並顯示有缺失值的欄位，這裡我們只在乎上面提到的重要欄位，恰好只有「開始時間」和「結束時間」需要處理

In [109]:
print(df.isnull().sum())
df[df.isnull()['開始時間']]

學年度           0
設立別           0
學校類別          0
學校代碼          0
學校名稱          0
學期別           0
開課單位          0
學制            0
課程代碼          0
課程名稱          0
必選修           0
學分數           0
起始日期          0
結束日期          0
星期            0
開始時間          1
結束時間          1
重複間隔          0
週次         4989
課程類別          0
授課教師          0
學程名稱及類別    4580
授課語言          0
上課地點        379
課程期程別         0
備註         2621
dtype: int64


,學年度,設立別,學校類別,學校代碼,學校名稱,學期別,開課單位,學制,課程代碼,課程名稱,...,結束時間,重複間隔,週次,課程類別,授課教師,學程名稱及類別,授課語言,上課地點,課程期程別,備註
4397,106,公立,一般大學,5,國立成功大學,2,管理學院,大學部,RZ30100Y,精實創業,...,NaN,每週,NaN,基礎,楊朝旭,NaN,中文,NaN,一般,上課教室技轉育成中心大會議室，上課時間詳如課程大綱


很幸運只有1列資料有缺失值，接著開始清理

In [110]:
print(f'before size: {df.shape}')
df.dropna(subset=['開始時間'], inplace=True)
print(f'after size: {df.shape}')

before size: (4989, 26)
after size: (4988, 26)


檢查重複值

In [111]:
repeated = df[df.duplicated(keep=False)]
print(f'重複 {len(repeated.index)} 列')
repeated.head(4)

重複 20 列


,學年度,設立別,學校類別,學校代碼,學校名稱,學期別,開課單位,學制,課程代碼,課程名稱,...,結束時間,重複間隔,週次,課程類別,授課教師,學程名稱及類別,授課語言,上課地點,課程期程別,備註
1051,106,公立,一般大學,5,國立成功大學,2,生命科學系學士班,大學部,C541100,生態學,...,12:00,每週,NaN,基礎,李亞夫*;邱慈暉 ;陳一菁 ;仲澤剛史,NaN,中文,生科系館 3404,一般,NaN
1052,106,公立,一般大學,5,國立成功大學,2,生命科學系學士班,大學部,C541100,生態學,...,10:00,每週,NaN,基礎,李亞夫*;邱慈暉 ;陳一菁 ;仲澤剛史,NaN,中文,生科系館 3404,一般,NaN
1065,106,公立,一般大學,5,國立成功大學,2,生命科學系學士班,大學部,C541100,生態學,...,12:00,每週,NaN,基礎,李亞夫*;邱慈暉 ;陳一菁 ;仲澤剛史,NaN,中文,生科系館 3404,一般,NaN
1066,106,公立,一般大學,5,國立成功大學,2,生命科學系學士班,大學部,C541100,生態學,...,10:00,每週,NaN,基礎,李亞夫*;邱慈暉 ;陳一菁 ;仲澤剛史,NaN,中文,生科系館 3404,一般,NaN


清理重複值，重複的列中只保留一列，因此清除掉10列

In [112]:
print(f'before size: {df.shape}')
df.drop_duplicates(keep='first', inplace=True)
print(f'after size: {df.shape}')

before size: (4988, 26)
after size: (4978, 26)


檢查看看哪些欄位沒有意義可以刪除

In [113]:
for col in df.columns:
    print(f'{col}\n{df[col].unique()}\n')

學年度
[106]

設立別
['公立']

學校類別
['一般大學']

學校代碼
[5]

學校名稱
['國立成功大學']

學期別
[2]

開課單位
['外國語言' '體育室' '軍訓室' '教育學程' '計網中心' '服務學習（三）' '基礎國文' '通識中心' '共同科目英語授課'
 '華語中心' '大一全校不分系學士學位學程' '文學院學士班' '中國文學系學士班' '外國語文學系學士班' '歷史學系學士班'
 '台灣文學系學士班' '數學系學士班' '物理學系學士班' '化學系學士班' '地球科學系學士班' '生命科學系學士班'
 '生物科技與產業科學系學士班' '法律學系學士班' '政治學系學士班' '經濟學系學士班' '心理學系學士班' '機械工程學系學士班'
 '電機工程學系學士班' '化學工程學系學士班' '資源工程學系學士班' '材料科學及工程學系學士班' '土木工程學系學士班' '建築學系學士班'
 '水利及海洋工程學系學士班' '工程科學系學士班' '能源國際學士學位學程學士班' '系統及船舶機電工程學系學士班' '都巿計劃學系學士班'
 '工業設計學系學士班' '航空太空工程學系學士班' '環境工程學系學士班' '測量及空間資訊學系學士班' '資訊工程學系學士班'
 '光電科學與工程學系學士班' '生物醫學工程學系學士班' '會計學系學士班' '統計學系學士班' '工業與資訊管理學系學士班'
 '企業管理學系學士班' '交通管理科學系學士班' '護理學系學士班' '醫學檢驗生物技術學系學士班' '醫學系學士班' '物理治療學系學士班'
 '職能治療學系學士班' '藥學系學士班' '中國文學系中國文學碩、博士班' '外國語文學系碩、博士班' '歷史學系碩、博士班'
 '藝術研究所碩士班' '台灣文學系碩、博士班' '考古學研究所' '戲劇碩士學位學程' '數學系應用數學碩、博士班' '物理學系碩、博士班'
 '化學系碩、博士班' '地球科學系碩、博士班' '生命科學系碩、博士班' '生物科技與產業科學研究所碩、博士班' '光電科學與工程學系碩、博士班'
 '太空與電漿科學研究所碩、博士班' '理學院研究所' '工程管理碩士在職專班' '機械工程學系碩、博士班' '電機工程學系碩、博士班'
 '化學工程學系碩、博士班' '資源工程學系碩、

刪除不必要的欄位

In [114]:
print(df.shape)
if '學制' in df.columns:
    df = df[df['學制'] == '大學部']
    
if '必選修' in df.columns:
    df = df[df['必選修'] == '必修']

df = df[['開課單位', '星期', '開始時間', '結束時間', '課程類別']]

print(df.shape)
df.columns

(4978, 26)
(2099, 5)


Index(['開課單位', '星期', '開始時間', '結束時間', '課程類別'], dtype='object')

In [138]:
df['開課單位'].unique()[0:10]

array(['外國語言', '基礎國文', '通識中心', '中國文學系學士班', '外國語文學系學士班', '歷史學系學士班',
       '台灣文學系學士班', '數學系學士班', '物理學系學士班', '化學系學士班'], dtype=object)

資料中存在不是通識中心也非各科系的開課單位，例如：體育室、華語中心、服務學習等

依據 [國立成功大學通識課程選修要點](http://cge.ncku.edu.tw/p/404-1007-7830.php?Lang=zh-tw#%E5%9C%8B%E7%AB%8B%E6%88%90%E5%8A%9F%E5%A4%A7%E5%AD%B8%E9%80%9A%E8%AD%98%E8%AA%B2%E7%A8%8B%E9%81%B8%E4%BF%AE%E8%A6%81%E9%BB%9E(108%E5%AD%B8%E5%B9%B4%E5%BA%A6%E8%B5%B7%E5%85%A5%E5%AD%B8%E7%94%9F%E9%81%A9%E7%94%A8))，通識課包含「基礎國文」、「外國語言」、「領域通識」及「融合通識」，對應到資料中的開課單位為**基礎國文**、**外國語言**及**通識中心**

因此其他非系所名稱也非通識的開課單位，都不是我們這次要討論的範圍，刪掉

In [122]:
print(df.shape)
for i in ['體育室', '教育學程', '服務學習（三）', '共同科目英語授課', '華語中心']:
    df = df[df['開課單位'] != i]
print(df.shape)

(1907, 5)
(1857, 5)


為了方便，將上課時間做標籤編碼(label encoding)，07:10-08:00 為第一節，08:10-09:00為 第二節，以此類推

In [123]:
for i in range(15):
    df.replace([f'{i+7:02d}:10', f'{i+8:02d}:00'], i, inplace=True)

print(df['開始時間'].unique())
print(df['結束時間'].unique())

[2 6 3 8 5 1 7 9 '未定' 4 10 0 11]
[3 7 4 9 5 2 10 8 '未定' 1 11 6 0 12 14 13]


將時間未定的課程刪掉

In [157]:
print(df.shape)
df = df[df['開始時間'] != '未定']
df = df[df['結束時間'] != '未定']
print(df.shape)

(1777, 5)
(1777, 5)


[2, 3]

In [164]:
def is_clashed(class1, class2):
    ''' 判斷兩堂課是否衝堂 (pd.Series, pd.Series) => bool '''
    class1_time = range(class1['開始時間'], class1['結束時間'] + 1)
    return class2['開始時間'] in class1_time or class2['結束時間'] in class1_time

print(df.loc[1, :])
print(df.loc[10, :])

is_clashed(df.loc[1, :], df.loc[1, :])

開課單位        外國語言
星期             一
開始時間           2
結束時間           3
課程類別    通識(外國語言)
Name: 1, dtype: object
開課單位        外國語言
星期             一
開始時間           6
結束時間           7
課程類別    通識(外國語言)
Name: 10, dtype: object


True

## 視覺化呈現
我們想知道以下幾個問題：
1. 通識課的時間分布
2. 各系必修的時間分布
3. 各系必修課撞了多少通識？

[回頂部](#成大通識與各系必修撞課分析)

In [136]:
df['開課單位'].value_counts().head(10)

通識中心            154
醫學系學士班          150
機械工程學系學士班       100
化學工程學系學士班        92
電機工程學系學士班        77
外國語言             67
外國語文學系學士班        56
基礎國文             42
材料科學及工程學系學士班     41
工業與資訊管理學系學士班     37
Name: 開課單位, dtype: int64

## 結果分析與討論
[回頂部](#成大通識與各系必修撞課分析)